In [15]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch.optim.lr_scheduler

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1,bias = False)
        self.batch_1 = nn.BatchNorm2d(8)
        self.drop_1 = nn.Dropout(0.25)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1,bias = False)
        self.batch_2 = nn.BatchNorm2d(16)
        self.drop_2 = nn.Dropout(0.25)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        #self.batch_1 = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 16, 3, padding=1,bias = False)
        self.batch_3 = nn.BatchNorm2d(16)
        self.drop_3 = nn.Dropout(0.25)
        
        self.conv4 = nn.Conv2d(16,32, 3, padding=1,bias = False)
        self.batch_4 = nn.BatchNorm2d(32)
        self.drop_4 = nn.Dropout(0.25)
        
        #self.pool2 = nn.MaxPool2d(2, 2)
        #self.batch_1 = nn.BatchNorm2d(16)
        
        self.conv5 = nn.Conv2d(32,16,3, padding=1,bias = False)
        self.batch_5 = nn.BatchNorm2d(16)
        self.drop_5 = nn.Dropout(0.25)
        
        self.conv6 = nn.Conv2d(16, 10, 1,bias = False)
        
        #self.batch_1 = nn.BatchNorm2d(16)
        #self.conv6 = nn.Conv2d(512, 1024, 3)
        
        self.conv7 = nn.Conv2d(10,10,7,bias = False)

    def forward(self, x):
        x = self.pool1(self.drop_1((F.relu(self.batch_1(self.conv1(x))))))
                       
        x = self.pool2(self.drop_2(F.relu(self.batch_2(self.conv2(x)))))
        
        x = self.drop_3(F.relu(self.batch_3((self.conv3(x)))))
        
        x = self.drop_4(F.relu(self.batch_4((self.conv4(x)))))
        
        x = self.drop_5(F.relu(self.batch_5((self.conv5(x)))))
        
        x = F.relu(self.conv6(x))
        
        x = self.conv7(x)
        
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [4]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              72
       BatchNorm2d-2            [-1, 8, 28, 28]              16
           Dropout-3            [-1, 8, 28, 28]               0
         MaxPool2d-4            [-1, 8, 14, 14]               0
            Conv2d-5           [-1, 16, 14, 14]           1,152
       BatchNorm2d-6           [-1, 16, 14, 14]              32
           Dropout-7           [-1, 16, 14, 14]               0
         MaxPool2d-8             [-1, 16, 7, 7]               0
            Conv2d-9             [-1, 16, 7, 7]           2,304
      BatchNorm2d-10             [-1, 16, 7, 7]              32
          Dropout-11             [-1, 16, 7, 7]               0
           Conv2d-12             [-1, 32, 7, 7]           4,608
      BatchNorm2d-13             [-1, 32, 7, 7]              64
          Dropout-14             [-1, 3

<ipython-input-3-e22a0f4cd152>:54: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [5]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


100%|██████████| 9912422/9912422 [00:00<00:00, 472188341.09it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 39317005.46it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 137810684.61it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 22815004.51it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [16]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [17]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.83)

for epoch in range(0, 20):
    print(f"EPOCH: {epoch+1}")
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 1


  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-3-e22a0f4cd152>:54: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.1510014533996582 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 22.86it/s]



Test set: Average loss: 0.0701, Accuracy: 9796/10000 (97.960%)

EPOCH: 2


loss=0.04157406836748123 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 22.00it/s]



Test set: Average loss: 0.0488, Accuracy: 9854/10000 (98.540%)

EPOCH: 3


...


Test set: Average loss: 0.0328, Accuracy: 9896/10000 (98.960%)

EPOCH: 19


loss=0.06890197098255157 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.14it/s]



Test set: Average loss: 0.0294, Accuracy: 9905/10000 (99.050%)

EPOCH: 20


loss=0.027895519509911537 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 23.12it/s]



Test set: Average loss: 0.0259, Accuracy: 9917/10000 (99.170%)

